# Imports

In [ ]:
from transformers import AutoTokenizer
import openvino.runtime as ov
import warnings
from pathlib import Path
import numpy as np
import time

# Transformers Serialization

[ONNX](https://onnx.ai/) is a representation format for deep learning models that allows AI developers to easily transfer models between different frameworks. It is hugely popular among deep learning tools, like PyTorch, Caffe2, Apache MXNet, Microsoft Cognitive Toolkit, and many others. The transformers model from Huggingface needs to be converted into ONNX format first, and then converted to OpenVINO IR format. This process to export a model to ONNX format is known as [Serialization](https://huggingface.co/docs/transformers/serialization)

In [ ]:
!python -m transformers.onnx -h

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
serialize_command = f"python -m transformers.onnx \
    -m {checkpoint} \
    --feature sequence-classification model/"
! $serialize_command

## Initializing The Tokenizer

Text preprocessing is the method of cleaning the input text to make it fit for being fed into the model. [Tokenization](https://towardsdatascience.com/tokenization-for-natural-language-processing-a179a891bad4) is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning. It involves cleaning of the data and assigning tokens or ids to the words where words are represented in a vector space where similar words have similar vectors which helps to understand the contexts of the sentence. We're making use of a [AutoTokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer) from Huggingface, which is basically a pretrained tokenizer.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=checkpoint
)

# Model Optimizer

[Model Optimizer](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) is a cross-platform command-line tool that facilitates the transition between training and deployment environments, performs static model analysis, and adjusts deep learning models for optimal execution on end-point target devices.

In [ ]:
onnx_model_path = "model.onnx"
MODEL_DIR = "model/"
MODEL_DIR = f"{MODEL_DIR}"
onnx_model_path = Path(MODEL_DIR) / onnx_model_path

optimizer_command = f"mo \
    --input_model {onnx_model_path} \
    --output_dir {MODEL_DIR} \
    --model_name {checkpoint} \
    --input input_ids,attention_mask \
    --input_shape [1,128],[1,128]"
! $optimizer_command

OpenVINO™ Runtime uses [Infer Request](https://docs.openvino.ai/latest/openvino_docs_OV_UG_Infer_request.html) mechanism which allows running models on different devices in asynchronous or synchronous manners. The model graph is sent as an argument to the OpenVINO API and an inference request is created. The default inference mode is AUTO but it can be changed according to requirement and hardwares available. You can explore the different inference modes and their usage [here.](https://docs.openvino.ai/latest/openvino_docs_Runtime_Inference_Modes_Overview.html)

In [ ]:
warnings.filterwarnings("ignore")
core = ov.Core()
ir_model_xml = str((Path(MODEL_DIR) / checkpoint).with_suffix(".xml"))
compiled_model = core.compile_model(ir_model_xml)
infer_request = compiled_model.create_infer_request()

Defining a softmax function to extract the prediction from the output of the IR format.

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Inference

Creating a generic inference function to read the input and infer the result into 2 classes: Positive or Negative.

In [ ]:
def infer(input_text):
    input_text = tokenizer(
        input_text,
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="np",
    )
    inputs = dict(input_text)
    label = {0: "NEGATIVE", 1: "POSITIVE"}
    result = infer_request.infer(inputs=inputs)
    for i in result.values():
        probability = np.argmax(softmax(i))
    return label[probability]

For a single input sentence

In [ ]:
input_text = "I had a wonderful day"
start_time = time.perf_counter()
result = infer(input_text)
end_time = time.perf_counter()
total_time = end_time - start_time
print("Label: ", result)
print("Total Time: ", "%.2f" % total_time, " seconds")

Read from a file

In [ ]:
start_time = time.perf_counter()
with open("data/sample.txt", "r") as f:
    input_text = f.readlines()
    for lines in input_text:
        print("User Input: ", lines)
        result = infer(lines)
        print("Label: ", result, "\n")
end_time = time.perf_counter()
total_time = end_time - start_time
print("Total Time: ", "%.2f" % total_time, " seconds")